In [ ]:
import pandas as pd
import numpy as np
import akshare as ak

stock_zh_a_tick_tx_js_df = ak.stock_zh_a_tick_tx_js(symbol="sz002358")

c:\Users\cyw\.conda\envs\stock\lib\site-packages\akshare\stock\stock_zh_a_tick_tx.py:27: UserWarning: 正在下载数据，请稍等
  warnings.warn("正在下载数据，请稍等")


In [9]:
import pandas as pd


def build_support_resistance_table(
    df: pd.DataFrame,
    trade_date: str,
    symbol: str,
    price_bin: float = 0.01,
    dense_ratio: float = 0.3,
    method: str = "amount"   # amount / volume
):
    data = df.copy()

    # 1. VWAP
    vwap = (data["成交价格"] * data["成交量"]).sum() / data["成交量"].sum()

    # 2. 价格分桶
    data["price_bin"] = (data["成交价格"] / price_bin).round(0) * price_bin
    data["price_bin"] = data["price_bin"].round(3)

    # 3. 选择用成交金额 or 成交量
    if method == "amount":
        profile = data.groupby("price_bin")["成交金额"].sum()
    else:
        profile = data.groupby("price_bin")["成交量"].sum()

    profile = profile.sort_values(ascending=False)

    # 4. 密集成交区
    total = profile.sum()
    dense_bins = profile[(profile.cumsum() / total) <= dense_ratio]

    dense_lower = dense_bins.index.min()
    dense_upper = dense_bins.index.max()

    # 5. 支撑 / 压力定义
    support_price = dense_lower
    resistance_price = dense_upper

    # 6. 结果表
    result = pd.DataFrame([{
        "trade_date": trade_date,
        "symbol": symbol,

        "support_price": support_price,
        "resistance_price": resistance_price,

        "vwap": round(vwap, 3),

        "dense_lower": dense_lower,
        "dense_upper": dense_upper,
        "dense_ratio": dense_ratio,

        "calc_method": f"{method}_profile",
        "price_bin": price_bin,

        "total_amount": data["成交金额"].sum(),
        "total_volume": data["成交量"].sum()
    }])

    return result


In [10]:
sr_df = build_support_resistance_table(
    df=stock_zh_a_tick_tx_js_df,
    trade_date="2026-02-05",
    symbol="600519",
    price_bin=0.01,
    dense_ratio=0.3,
    method="amount"
)

print(sr_df)


   trade_date  symbol  support_price  resistance_price   vwap  dense_lower  \
0  2026-02-05  600519           7.55               7.6  7.547         7.55   

   dense_upper  dense_ratio     calc_method  price_bin  total_amount  \
0          7.6          0.3  amount_profile       0.01     275623770   

   total_volume  
0        365353  
